<a href="https://colab.research.google.com/github/raz0208/ModernBERT/blob/main/ModernBERT_TokenEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import required libraries
import os
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
# Load ModernBERT tokenizer and model from Hugging Face
MODEL_NAME = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

In [3]:
# Function to gest text and return the embedding
def get_text_embedding(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the embeddings (use CLS token for sentence-level embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: [batch_size, hidden_size]

    return cls_embedding.squeeze().numpy()

In [4]:
# Function to get per-token embeddings
def get_token_embeddings(text):
    # Tokenize with return_tensors and also get token strings
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, return_attention_mask=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state.squeeze(0)  # shape: [seq_len, hidden_size]

    # Prepare cleaned token list and their embeddings
    token_embeddings = []
    for token, emb in zip(tokens, embeddings):
        # Skip special tokens and punctuation/whitespace
        if token in tokenizer.all_special_tokens:
            continue
        if token.startswith("▁"):  # For SentencePiece or models like ModernBERT using underscores
            token = token[1:]  # Remove prefix
        if not token.isalnum():  # Skip non-alphanumeric tokens
            continue
        token_embeddings.append((token, emb.numpy()))

    return token_embeddings

In [5]:
# Example usage (Sentence: This is an application about Breast Cancer.)
if __name__ == "__main__":
    user_text = input("Enter your text: ")

    # Get sentence embedding
    embedding = get_text_embedding(user_text)
    print("\nSentence Embedding vector shape:", embedding.shape)
    print("Sentence Embedding (first 10 values):", embedding[:10])

    # Get per-token embeddings
    token_embeddings = get_token_embeddings(user_text)
    print("\nToken-wise Embeddings:")
    for token, emb in token_embeddings:
        print(f"Token: {token:15} | Embedding (first 5 vals): {emb[:5]}")

Enter your text: This is an application about Breast Cancer.

Sentence Embedding vector shape: (768,)
Sentence Embedding (first 10 values): [ 0.42236355 -0.8862073  -0.6536694  -0.2981413  -0.5874422  -0.720903
 -0.8588484  -0.89695704  0.5856571  -0.9214181 ]

Token-wise Embeddings:
Token: This            | Embedding (first 5 vals): [-0.6776887  -0.92234474 -0.38191086  0.1988687   0.6651754 ]
Token: Ġis             | Embedding (first 5 vals): [ 0.55707484 -1.9195614  -0.5415806   0.16398725  0.47974288]
Token: Ġan             | Embedding (first 5 vals): [ 0.8371718  -1.4686286  -0.6741889   0.5709336  -0.00925406]
Token: Ġapplication    | Embedding (first 5 vals): [ 0.34761772 -2.8377428   0.42626727 -0.28745332  1.098285  ]
Token: Ġabout          | Embedding (first 5 vals): [ 1.9028491  -1.1237415  -0.74431485 -0.17381278 -0.2998082 ]
Token: ĠBreast         | Embedding (first 5 vals): [ 1.344319   -1.9794563  -1.4910724   0.20805806  0.18633802]
Token: ĠCancer         | Embedding (f